In [ ]:
'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
'''

In [1]:
import pandas as pd
styles_multiTags = pd.read_csv("../input/multitags/styles_multiTags.csv")
styles_multiTags = styles_multiTags[styles_multiTags.id != 39410]
styles_multiTags = styles_multiTags[styles_multiTags.id != 12347]
styles_multiTags = styles_multiTags[styles_multiTags.id != 39041]
styles_multiTags = styles_multiTags[styles_multiTags.id != 39403]
styles_multiTags = styles_multiTags[styles_multiTags.id != 39401]
styles_multiTags = styles_multiTags[styles_multiTags.id != 39425]
styles_multiTags43500 = styles_multiTags[:43500]
styles_multiTags43500.head()

,Unnamed: 0,id,tags
0,0,15970,"Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual"
1,1,39386,"Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual"
2,2,59263,"Women,Accessories,Watches,Watches,Silver,Winte..."
3,3,21379,"Men,Apparel,Bottomwear,Track Pants,Black,Fall,..."
4,4,53759,"Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual"


In [ ]:
styles_multiTags.tail()

In [ ]:
import os
filenames = os.listdir('/kaggle/input/fashion-product-images-dataset/fashion-dataset/images')
for value in styles_multiTags['id'].values:
    img = str(value)+".jpg"
    if img not in filenames:
        print(value)
        


In [ ]:
type(filenames)

In [ ]:
from fastai.vision import *
path = Path('../input/')
data = ImageList.from_csv(path, 'multitags/styles_multiTags.csv', cols=1, folder='fashion-product-images-dataset/fashion-dataset/images/', suffix='.jpg')
data

In [ ]:
path1 = Path('../input/fashion-product-images-dataset/fashion-dataset/images/')
data1 = ImageList.from_df(styles_multiTags43500, path1, cols=1, suffix='.jpg')
data1

In [ ]:
data1 = data1.split_by_rand_pct(0.1).label_from_df(cols=2, label_delim=',').transform(get_transforms(), size=224, resize_method=3).databunch(bs=64).normalize(imagenet_stats)
#data1 = data1.split_by_rand_pct(0.1).label_from_df(cols=2, label_delim=',').transform(get_transforms(), size=512, resize_method=3).databunch(bs=64).normalize(imagenet_stats)

In [ ]:
data1

In [ ]:
data1.train_ds.x

In [ ]:
data1.train_ds.y

In [ ]:
data1.valid_ds

In [ ]:
data1.show_batch(rows=3, figsize=(20,12))

In [ ]:
data1.classes

In [ ]:
torch.cuda.is_available()

In [ ]:
#BEFORE THIS STEP, Click on Add Data and Search for Resnet152 from Kaggle. Because Kaggle serves read-only dirs, we cannot download pre-trained weights for Resnet. Now we have to make a dir 
#for copying those weights and hence we are making ~/.torch/models folder. 
cache_dir = os.path.expanduser(os.path.join('/root/.cache/','torch/checkpoints'))
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir) #first make ~/.torch if not already available.


In [ ]:
models_dir = os.path.join(cache_dir,'models')
if not os.path.exists(models_dir):
    os.makedirs(models_dir) #then make ~/.torch/models, if not already available. 

In [ ]:
#Copied resnet34.pth, which are pretrained weights on Resnet34 to our folder into resnet<version>-<sha-hash>.pth
!cp ../input/resnet152/resnet152-b121ed2d.pth /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth



In [ ]:
#MODEL_PATH is declared this way and glued to model_dir attr of cnn_learner.
#MODEL_PATH = '/tmp/models'
#model_dir=MODEL_PATH
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = cnn_learner(data1, models.resnet152, metrics=[acc_02,f_score])
learn.model_dir = "/kaggle/working"

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.show_results(rows=2, figsize=(24,15))

In [ ]:
styles_multiTags.tail()

In [ ]:
img = open_image('../input/fashion-product-images-dataset/fashion-dataset/images/34411.jpg')
img = img.apply_tfms(tfms=get_transforms()[1], size=224, resize_method=3) # resize the images the same way a the pre-processing
preds, idx, output = learn.predict(img)
preds

In [ ]:
d = dict({learn.data.classes[i]: round(to_np(p)*100,2) for i, p in enumerate(output) if p > 0.2})
img.show(title=str(d))

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-06,1e-04))

In [ ]:
'''
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE
'''

In [ ]:
'''
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)
'''

In [ ]:
'''
IMAGE_SIZE = [512, 512]
EPOCHS = 12
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
STEPS_PER_EPOCH = len(data1.train_ds)// BATCH_SIZE
'''

In [ ]:
'''
with strategy.scope():
    #pretrained_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.Xception(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = False # False = transfer learning, True = fine-tuning
    
    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(data1.classes), activation='softmax')
    ])
        
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model.summary()
'''

In [ ]:
#history = model.fit(data1.train_ds.x, data1.train_ds.y, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, validation_data=(data1.valid_ds.x, data1.valid_ds.y))